In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from random import randint

In [56]:
df = pd.read_csv('memphis_restaurants.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [60]:
for row in df.iterrows():
    try:
        #### Get the business name and url ####
        business_name = row[1]['name']
        business_url = row[1]['url']
        print(business_name, business_url)

        #### Request the page ####
        html = requests.get(business_url)
        soup = BeautifulSoup(html.content, 'html.parser')
        
        #### Average Rating ####
        try:
            mean_rating = soup.select('.i-stars--large-5__373c0__1GcGD')
            mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
        except:
            try:
                mean_rating = soup.select('.i-stars--large-4-half__373c0__2lYkD')
                mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
            except:
                try:
                    mean_rating = soup.select('.i-stars--large-4__373c0__1d6HV')
                    mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
                except:
                    try:
                        mean_rating = soup.select('.i-stars--large-3-half__373c0__2z4jR')
                        mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
                    except:
                        try: 
                            mean_rating = soup.select('.i-stars--large-3__373c0__3_Jon')
                            mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
                        except:
                            mean_rating = 0


        # Total Number of Reviews
        total_reviews = soup.select('.arrange-unit-fill__373c0__17z0h.nowrap__373c0__1_N1j .css-bq71j2')
        total_reviews = int(total_reviews[0].string.split(' ')[0])

        # Claimed Status
        try:
            claimed_status = soup.select('.css-1xxismk .css-166la90')[0].string
            claimed_status = 0
        except:
            claimed_status = 1

        # Price Category
        try:
            price_cat = soup.select('.margin-r1__373c0__zyKmV+ .border-color--default__373c0__2oFDT .css-1xxismk')
            price_cat = len((price_cat[0].text).strip())
        except:
            price_cat = None
            
        # Restaurant Categories
        categories = soup.select('.css-bq71j2 .css-166la90')
        categories = [x.string for x in categories]

        # Total number of Photos
        try:
            tot_n_photos = soup.select('.css-1fepc68 .css-ardur')
            tot_n_photos = int(tot_n_photos[0].string.split(' ')[1])
        except:
            tot_n_photos = 0
            
        # Health Score
        try:
            health_score = soup.select('.label-spacing-v2__373c0__PBYkt')
            health_score = int(health_score[0].string.split(' ')[0])
        except:
            health_score = None
            
        # Covid Updates
        cov_updates = soup.select('.margin-b1__373c0__1khoT .css-1h1j0y3')
        cov_updates = [x.string for x in cov_updates]

        # Street Address
        st_address = soup.select('.css-1bmgof7 .raw__373c0__3rcx7')
        st_address = st_address[0].string    

        business_info = [business_name, business_url, total_reviews, claimed_status,
                             price_cat, categories, tot_n_photos, health_score, cov_updates, st_address]

        #### Get the data for multiple review pages ####
        #### Set the number of review pages that should be scraped ####

        for i in range(3):
            try:
                url2 = business_url + '?start=' + str(i * 10)
                html = requests.get(url2)
                soup = BeautifulSoup(html.content, 'html.parser')

                #### Names
                names = soup.select('.css-m6anxm .css-166la90')
                review_names = []

                for name in names:
                    review_names.append(name.string)

                #### Ratings
                ratings = soup.select('.margin-b1-5__373c0__2Wblx .overflow--hidden__373c0__2B0kz')

                review_ratings = []

                for rating in ratings:
                    review_ratings.append(float(re.sub(' star rating', 
                                                       '',rating.attrs['aria-label'])))
                #### Dates
                dates = soup.select('.margin-b1-5__373c0__2Wblx .css-e81eai')

                review_dates = []

                for date in dates:
                    review_dates.append(date.string)

                #### Text
                texts = soup.select('.css-n6i4z7 .raw__373c0__3rcx7')
                review_texts = []

                for text in texts:
                    review_texts.append(text.get_text())

                #### Reviewer's Location
                reviewer_loc = soup.select('.arrange-unit-fill__373c0__17z0h .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .css-n6i4z7')

                reviewer_city = []
                reviewer_state = []

                for loc in reviewer_loc:
                    if len(loc.get_text().split(',')) == 2:
                        reviewer_city.append(loc.get_text().split(',')[0])
                        reviewer_state.append(loc.get_text().split(',')[1])
                    elif len(loc.get_text().split(',')) == 3:
                        reviewer_city.append(loc.get_text().split(',')[1])
                        reviewer_state.append(loc.get_text().split(',')[2])
                    else:
                        reviewer_city.append(None)
                        reviewer_city.append(None)    

                #### Votes
                votes = soup.select('.arrange-unit-fill__373c0__17z0h .display--inline__373c0__1DbOG .css-1ha1j8d')

                useful_votes = []
                funny_votes = []
                cool_votes = []

                for vote in votes:
                    if vote.get_text().split(' ')[0] == 'Useful':
                        try:
                            useful_votes.append(vote.get_text().split(' ')[1])
                        except:
                            useful_votes.append(0)
                    elif vote.get_text().split(' ')[0] == 'Funny':
                        try:
                            funny_votes.append(vote.get_text().split(' ')[1])
                        except:
                            funny_votes.append(0)
                    else:
                        try:
                            cool_votes.append(vote.get_text().split(' ')[1])
                        except:
                            cool_votes.append(0)    
                review_vars1 = [review_names, review_ratings, review_dates, 
                                review_texts, reviewer_city, reviewer_state,
                                useful_votes, funny_votes, cool_votes]


                #### Getting the inconsistent variables ####
                n_photos = []
                r_friends = []
                r_reviews = []
                r_photos = []
                elite_tag = []

                raw_reviews = soup.select('.review__373c0__13kpL')
                for raw_review in raw_reviews:

                    ### Number of Photos in the review ###
                    raw_n_photos = raw_review.select('.margin-b0-5__373c0__Fo75u:nth-child(1) .css-166la90')
                    if len(raw_n_photos) == 0:
                        n_photos.append(0)
                    else:
                        try:
                            n_photos.append(int(raw_n_photos[0].string.split()[0]))
                        except:
                            n_photos.append(0)

                    ### Profile tags (# Friends, # Reviews, # Pictures) ###   
                    raw_tags = raw_review.select('.arrange-unit-fill__373c0__17z0h .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .css-1dgkz3l')
                    r_friends.append(int(raw_tags[0].string))
                    r_reviews.append(int(raw_tags[1].string))
                    try:
                        r_photos.append(int(raw_tags[2].string))
                    except:
                        r_photos.append(0)

                    ### The "Yelp Elite Squad" tag ###
                    raw_elite_tag = raw_review.select('.css-1hx6l2b')
                    if len(raw_elite_tag) == 0:
                        elite_tag.append(0)
                    else:
                        elite_tag.append(1)

                #### Combining all variables into 1 dataset ####

                business_names = [business_name] * len(review_names)
                business_urls = [business_url] * len(review_names)
                mean_ratings = [mean_rating] * len(review_names)
                total_reviews_all = [total_reviews] * len(review_names)
                claimed_status_all = [claimed_status] * len(review_names)
                price_cat_all = [price_cat] * len(review_names)
                categories_all = [categories] * len(review_names)
                tot_n_photos_all = [tot_n_photos] * len(review_names)
                health_score_all = [health_score] * len(review_names)
                cov_updates_all = [cov_updates] * len(review_names)
                st_address_all = [st_address] * len(review_names)

                review_data = list(zip(business_names, business_urls, mean_ratings, total_reviews_all, 
                                       claimed_status_all,price_cat_all, categories_all, 
                                       tot_n_photos_all, health_score_all, cov_updates_all,
                                       st_address_all, review_names, review_ratings, review_dates, 
                                       review_texts, n_photos, r_friends, r_reviews, 
                                       r_photos, elite_tag))

                df_reviews = pd.DataFrame(data=review_data,
                                          columns = ['name_business', 'url', 'mean_rating', 'total_reviews', 
                                                     'claimed_status','price_cat', 'categories', 
                                                     'tot_n_photos', 'health_score', 'cov_updates', 
                                                     'st_address','username', 'rating', 
                                                     'date_review', 'text', 'n_photos',
                                                     'r_friends', 'r_reviews', 'r_photos',
                                                     'elite_tag'])

                with open('memphis_reviews.csv', 'a', newline='', encoding='utf-8') as f:
                    df_reviews.to_csv(f, index=False, header=False)

                print('{} review page {} has been scraped'.format(row[1]['name'], str(i)))
                time.sleep(randint(4, 12))
                
            except:
                print('Error in stage 2')

        print('{} has been scraped'.format(row[1]['name']))    
        time.sleep(randint(4, 12))
    
    except:
        print('Error in stage 1')

The Second Line https://www.yelp.com/biz/the-second-line-memphis
The Second Line review page 0 has been scraped
The Second Line review page 1 has been scraped
The Second Line has been scraped


### Test version without try/except to find the errors

In [7]:
#### Get the business name and url ####
business_name = 'The Second Line'
business_url = 'https://www.yelp.com/biz/the-second-line-memphis'
print(business_name, business_url)

#### Request the page ####
html = requests.get(business_url)
soup = BeautifulSoup(html.content, 'html.parser')

# Total Number of Reviews
total_reviews = soup.select('.arrange-unit-fill__373c0__17z0h.nowrap__373c0__1_N1j .css-bq71j2')
total_reviews = int(total_reviews[0].string.split(' ')[0])

# Claimed Status
try:
    claimed_status = soup.select('.css-1xxismk .css-166la90')[0].string
    claimed_status = 0
except:
    claimed_status = 1

# Price Category
try:
    price_cat = soup.select('.margin-r1__373c0__zyKmV+ .border-color--default__373c0__2oFDT .css-1xxismk')
    price_cat = len((price_cat[0].text).strip())
except:
    price_cat = None
# Restaurant Categories
categories = soup.select('.css-bq71j2 .css-166la90')
categories = [x.string for x in categories]

# Total number of Photos
tot_n_photos = soup.select('.css-1fepc68 .css-ardur')
tot_n_photos = int(tot_n_photos[0].string.split(' ')[1])

# Health Score
try:
    health_score = soup.select('.label-spacing-v2__373c0__PBYkt')
    health_score = int(health_score[0].string.split(' ')[0])
except:
    health_score = None

# Covid Updates
cov_updates = soup.select('.margin-b1__373c0__1khoT .css-1h1j0y3')
cov_updates = [x.string for x in cov_updates]

# Street Address
st_address = soup.select('.css-1bmgof7 .raw__373c0__3rcx7')
st_address = st_address[0].string    

#business_info = [business_name, business_url, total_reviews, claimed_status,
 #                    price_cat, categories, tot_n_photos, health_score, cov_updates, st_address]

#     for x in business_info:
#         print(x)

#### Get the data for multiple review pages ####
#### Set the number of review pages that should be scraped ####

for i in [2]:

    url2 = business_url + '?start=' + str(i * 10)
    html = requests.get(url2)
    soup = BeautifulSoup(html.content, 'html.parser')

    #### Names
    names = soup.select('.css-m6anxm .css-166la90')
    review_names = []

    for name in names:
        review_names.append(name.string)

    #### Ratings
    ratings = soup.select('.margin-b1-5__373c0__2Wblx .overflow--hidden__373c0__2B0kz')

    review_ratings = []

    for rating in ratings:
        review_ratings.append(float(re.sub(' star rating', 
                                           '',rating.attrs['aria-label'])))
    #### Dates
    dates = soup.select('.margin-b1-5__373c0__2Wblx .css-e81eai')

    review_dates = []

    for date in dates:
        review_dates.append(date.string)

    #### Text
    texts = soup.select('.css-n6i4z7 .raw__373c0__3rcx7')
    review_texts = []

    for text in texts:
        review_texts.append(text.get_text())

    #### Reviewer's Location
    reviewer_loc = soup.select('.arrange-unit-fill__373c0__17z0h .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .css-n6i4z7')

    reviewer_city = []
    reviewer_state = []

    for loc in reviewer_loc:
        if len(loc.get_text().split(',')) == 2:
            reviewer_city.append(loc.get_text().split(',')[0])
            reviewer_state.append(loc.get_text().split(',')[1])
        elif len(loc.get_text().split(',')) == 3:
            reviewer_city.append(loc.get_text().split(',')[1])
            reviewer_state.append(loc.get_text().split(',')[2])
        else:
            reviewer_city.append(None)
            reviewer_city.append(None)    

    #### Votes
    votes = soup.select('.arrange-unit-fill__373c0__17z0h .display--inline__373c0__1DbOG .css-1ha1j8d')

    useful_votes = []
    funny_votes = []
    cool_votes = []

    for vote in votes:
        if vote.get_text().split(' ')[0] == 'Useful':
            try:
                useful_votes.append(vote.get_text().split(' ')[1])
            except:
                useful_votes.append(0)
        elif vote.get_text().split(' ')[0] == 'Funny':
            try:
                funny_votes.append(vote.get_text().split(' ')[1])
            except:
                funny_votes.append(0)
        else:
            try:
                cool_votes.append(vote.get_text().split(' ')[1])
            except:
                cool_votes.append(0)    
    review_vars1 = [review_names, review_ratings, review_dates, 
                    review_texts, reviewer_city, reviewer_state,
                    useful_votes, funny_votes, cool_votes]


    #### Getting the inconsistent variables ####
    n_photos = []
    r_friends = []
    r_reviews = []
    r_photos = []
    elite_tag = []

    raw_reviews = soup.select('.review__373c0__13kpL')
    for raw_review in raw_reviews:

        ### Number of Photos in the review ###
        raw_n_photos = raw_review.select('.margin-b0-5__373c0__Fo75u:nth-child(1) .css-166la90')
        if len(raw_n_photos) == 0:
            n_photos.append(0)
        else:
            try:
                n_photos.append(int(raw_n_photos[0].string.split()[0]))
            except:
                n_photos.append(0)

        ### Profile tags (# Friends, # Reviews, # Pictures) ###   
        raw_tags = raw_review.select('.arrange-unit-fill__373c0__17z0h .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .css-1dgkz3l')
        r_friends.append(int(raw_tags[0].string))
        r_reviews.append(int(raw_tags[1].string))
        try:
            r_photos.append(int(raw_tags[2].string))
        except:
            r_photos.append(0)

        ### The "Yelp Elite Squad" tag ###
        raw_elite_tag = raw_review.select('.css-1hx6l2b')
        if len(raw_elite_tag) == 0:
            elite_tag.append(0)
        else:
            elite_tag.append(1)

    #print(n_photos, r_photos, elite_tag) 

    #### Combining all variables into 1 dataset ####

    business_names = [business_name] * len(review_names)
    business_urls = [business_url] * len(review_names)
    total_reviews_all = [total_reviews] * len(review_names)
    claimed_status_all = [claimed_status] * len(review_names)
    price_cat_all = [price_cat] * len(review_names)
    categories_all = [categories] * len(review_names)
    tot_n_photos_all = [tot_n_photos] * len(review_names)
    health_score_all = [health_score] * len(review_names)
    cov_updates_all = [cov_updates] * len(review_names)
    st_address_all = [st_address] * len(review_names)

    review_data = list(zip(business_names, business_urls, total_reviews_all, 
                           claimed_status_all,price_cat_all, categories_all, 
                           tot_n_photos_all, health_score_all, cov_updates_all,
                           st_address_all, review_names, review_ratings, review_dates, 
                           review_texts, n_photos, r_friends, r_reviews, 
                           r_photos, elite_tag))

    df_reviews = pd.DataFrame(data=review_data,
                              columns = ['name_business', 'url', 'total_reviews', 
                                         'claimed_status','price_cat', 'categories', 
                                         'tot_n_photos', 'health_score', 'cov_updates', 
                                         'st_address','username', 'rating', 
                                         'date_review', 'text', 'n_photos',
                                         'r_friends', 'r_reviews', 'r_photos',
                                         'elite_tag'])

    with open('cleantest2.csv', 'a', newline='', encoding='utf-8') as f:
        df_reviews.to_csv(f, index=False, header=False)

    #print('{} review page {} has been scraped'.format(row[1]['name'], str(i)))
    time.sleep(randint(2, 10))


#print('{} has been scraped'.format(row[1]['name']))    



The Second Line https://www.yelp.com/biz/the-second-line-memphis


#### After getting blocked on Yelp, we can continue the scraper where it left off by looking at the last business in the csv, and continuing from the next business in the master list.

In [53]:
df_reviews = pd.read_csv('memphis_reviews.csv', header=None, encoding='utf-8')
idx = df.index[df['name'] == df_reviews.iloc[-1][0]]
df_continue = df[(idx[0]+1):]

In [54]:
for row in df_small.iterrows():
    try:
        #### Get the business name and url ####
        business_name = row[1]['name']
        business_url = row[1]['url']
        print(business_name, business_url)

        #### Request the page ####
        html = requests.get(business_url)
        soup = BeautifulSoup(html.content, 'html.parser')
        
        #### Average Rating ####
        try:
            mean_rating = soup.select('.i-stars--large-5__373c0__1GcGD')
            mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
        except:
            try:
                mean_rating = soup.select('.i-stars--large-4-half__373c0__2lYkD')
                mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
            except:
                try:
                    mean_rating = soup.select('.i-stars--large-4__373c0__1d6HV')
                    mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
                except:
                    try:
                        mean_rating = soup.select('.i-stars--large-3-half__373c0__2z4jR')
                        mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
                    except:
                        try: 
                            mean_rating = soup.select('.i-stars--large-3__373c0__3_Jon')
                            mean_rating = float(mean_rating[0].attrs['aria-label'].split(' ')[0])
                        except:
                            mean_rating = 0


        # Total Number of Reviews
        total_reviews = soup.select('.arrange-unit-fill__373c0__17z0h.nowrap__373c0__1_N1j .css-bq71j2')
        total_reviews = int(total_reviews[0].string.split(' ')[0])

        # Claimed Status
        try:
            claimed_status = soup.select('.css-1xxismk .css-166la90')[0].string
            claimed_status = 0
        except:
            claimed_status = 1

        # Price Category
        try:
            price_cat = soup.select('.margin-r1__373c0__zyKmV+ .border-color--default__373c0__2oFDT .css-1xxismk')
            price_cat = len((price_cat[0].text).strip())
        except:
            price_cat = None
            
        # Restaurant Categories
        categories = soup.select('.css-bq71j2 .css-166la90')
        categories = [x.string for x in categories]

        # Total number of Photos
        try:
            tot_n_photos = soup.select('.css-1fepc68 .css-ardur')
            tot_n_photos = int(tot_n_photos[0].string.split(' ')[1])
        except:
            tot_n_photos = 0
            
        # Health Score
        try:
            health_score = soup.select('.label-spacing-v2__373c0__PBYkt')
            health_score = int(health_score[0].string.split(' ')[0])
        except:
            health_score = None
            
        # Covid Updates
        cov_updates = soup.select('.margin-b1__373c0__1khoT .css-1h1j0y3')
        cov_updates = [x.string for x in cov_updates]

        # Street Address
        st_address = soup.select('.css-1bmgof7 .raw__373c0__3rcx7')
        st_address = st_address[0].string    

        business_info = [business_name, business_url, total_reviews, claimed_status,
                             price_cat, categories, tot_n_photos, health_score, cov_updates, st_address]

        #### Get the data for multiple review pages ####
        #### Set the number of review pages that should be scraped ####

        for i in range(2):
            try:
                url2 = business_url + '?start=' + str(i * 10)
                html = requests.get(url2)
                soup = BeautifulSoup(html.content, 'html.parser')

                #### Names
                names = soup.select('.css-m6anxm .css-166la90')
                review_names = []

                for name in names:
                    review_names.append(name.string)

                #### Ratings
                ratings = soup.select('.margin-b1-5__373c0__2Wblx .overflow--hidden__373c0__2B0kz')

                review_ratings = []

                for rating in ratings:
                    review_ratings.append(float(re.sub(' star rating', 
                                                       '',rating.attrs['aria-label'])))
                #### Dates
                dates = soup.select('.margin-b1-5__373c0__2Wblx .css-e81eai')

                review_dates = []

                for date in dates:
                    review_dates.append(date.string)

                #### Text
                texts = soup.select('.css-n6i4z7 .raw__373c0__3rcx7')
                review_texts = []

                for text in texts:
                    review_texts.append(text.get_text())

                #### Reviewer's Location
                reviewer_loc = soup.select('.arrange-unit-fill__373c0__17z0h .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .css-n6i4z7')

                reviewer_city = []
                reviewer_state = []

                for loc in reviewer_loc:
                    if len(loc.get_text().split(',')) == 2:
                        reviewer_city.append(loc.get_text().split(',')[0])
                        reviewer_state.append(loc.get_text().split(',')[1])
                    elif len(loc.get_text().split(',')) == 3:
                        reviewer_city.append(loc.get_text().split(',')[1])
                        reviewer_state.append(loc.get_text().split(',')[2])
                    else:
                        reviewer_city.append(None)
                        reviewer_city.append(None)    

                #### Votes
                votes = soup.select('.arrange-unit-fill__373c0__17z0h .display--inline__373c0__1DbOG .css-1ha1j8d')

                useful_votes = []
                funny_votes = []
                cool_votes = []

                for vote in votes:
                    if vote.get_text().split(' ')[0] == 'Useful':
                        try:
                            useful_votes.append(vote.get_text().split(' ')[1])
                        except:
                            useful_votes.append(0)
                    elif vote.get_text().split(' ')[0] == 'Funny':
                        try:
                            funny_votes.append(vote.get_text().split(' ')[1])
                        except:
                            funny_votes.append(0)
                    else:
                        try:
                            cool_votes.append(vote.get_text().split(' ')[1])
                        except:
                            cool_votes.append(0)    
                review_vars1 = [review_names, review_ratings, review_dates, 
                                review_texts, reviewer_city, reviewer_state,
                                useful_votes, funny_votes, cool_votes]


                #### Getting the inconsistent variables ####
                n_photos = []
                r_friends = []
                r_reviews = []
                r_photos = []
                elite_tag = []

                raw_reviews = soup.select('.review__373c0__13kpL')
                for raw_review in raw_reviews:

                    ### Number of Photos in the review ###
                    raw_n_photos = raw_review.select('.margin-b0-5__373c0__Fo75u:nth-child(1) .css-166la90')
                    if len(raw_n_photos) == 0:
                        n_photos.append(0)
                    else:
                        try:
                            n_photos.append(int(raw_n_photos[0].string.split()[0]))
                        except:
                            n_photos.append(0)

                    ### Profile tags (# Friends, # Reviews, # Pictures) ###   
                    raw_tags = raw_review.select('.arrange-unit-fill__373c0__17z0h .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .border-color--default__373c0__2oFDT .css-1dgkz3l')
                    r_friends.append(int(raw_tags[0].string))
                    r_reviews.append(int(raw_tags[1].string))
                    try:
                        r_photos.append(int(raw_tags[2].string))
                    except:
                        r_photos.append(0)

                    ### The "Yelp Elite Squad" tag ###
                    raw_elite_tag = raw_review.select('.css-1hx6l2b')
                    if len(raw_elite_tag) == 0:
                        elite_tag.append(0)
                    else:
                        elite_tag.append(1)

                #### Combining all variables into 1 dataset ####

                business_names = [business_name] * len(review_names)
                business_urls = [business_url] * len(review_names)
                mean_ratings = [mean_rating] * len(review_names)
                total_reviews_all = [total_reviews] * len(review_names)
                claimed_status_all = [claimed_status] * len(review_names)
                price_cat_all = [price_cat] * len(review_names)
                categories_all = [categories] * len(review_names)
                tot_n_photos_all = [tot_n_photos] * len(review_names)
                health_score_all = [health_score] * len(review_names)
                cov_updates_all = [cov_updates] * len(review_names)
                st_address_all = [st_address] * len(review_names)

                review_data = list(zip(business_names, business_urls, mean_ratings, total_reviews_all, 
                                       claimed_status_all,price_cat_all, categories_all, 
                                       tot_n_photos_all, health_score_all, cov_updates_all,
                                       st_address_all, review_names, review_ratings, review_dates, 
                                       review_texts, n_photos, r_friends, r_reviews, 
                                       r_photos, elite_tag))

                df_reviews = pd.DataFrame(data=review_data,
                                          columns = ['name_business', 'url', 'mean_rating', 'total_reviews', 
                                                     'claimed_status','price_cat', 'categories', 
                                                     'tot_n_photos', 'health_score', 'cov_updates', 
                                                     'st_address','username', 'rating', 
                                                     'date_review', 'text', 'n_photos',
                                                     'r_friends', 'r_reviews', 'r_photos',
                                                     'elite_tag'])

                with open('memphis_reviews4.csv', 'a', newline='', encoding='utf-8') as f:
                    df_reviews.to_csv(f, index=False, header=False)

                print('{} review page {} has been scraped'.format(row[1]['name'], str(i)))
                time.sleep(randint(4, 12))
                
            except:
                print('Error in stage 2')

        print('{} has been scraped'.format(row[1]['name']))    
        time.sleep(randint(4, 12))
    
    except:
        print('Error in stage 1')

Chez Philippe https://www.yelp.com/biz/chez-philippe-memphis
Error in stage 1
Flying Fish https://www.yelp.com/biz/flying-fish-memphis-2
Error in stage 1
Pete & Sam’s Restaurant https://www.yelp.com/biz/pete-and-sams-restaurant-memphis
Error in stage 1
Orange Mound Grill https://www.yelp.com/biz/orange-mound-grill-memphis
Error in stage 1
Huey’s - Midtown https://www.yelp.com/biz/hueys-midtown-memphis-4
Error in stage 1
Dino’s Grill https://www.yelp.com/biz/dinos-grill-memphis
Error in stage 1
Ciao Bella Italian Grill and Bar https://www.yelp.com/biz/ciao-bella-italian-grill-and-bar-memphis
Error in stage 1
Freddy’s Frozen Custard & Steakburgers https://www.yelp.com/biz/freddys-frozen-custard-and-steakburgers-memphis-5
Error in stage 1
Bala’s Bistro https://www.yelp.com/biz/balas-bistro-memphis-2
Error in stage 1
Venice Kitchen https://www.yelp.com/biz/venice-kitchen-memphis-2
Error in stage 1
Marciano https://www.yelp.com/biz/marciano-memphis
Error in stage 1
Dixie Queen https://www.y